In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Chocolate Sales').getOrCreate()

In [ ]:
# Read the dataset
df_spark = spark.read.option("header", "true").csv("../data/Chocolate Sales.csv", inferSchema = True)

# Another option to read dataset
spark.read.csv("../data/Chocolate Sales.csv", header=True, inferSchema=True).show()

## 1. Introduction

In [ ]:
# Check the schema
df_spark.printSchema()

In [ ]:
# Get all the column names
df_spark.columns

In [ ]:
# Check the first 10 rows
df_spark.head(10)

In [ ]:
# Select only the Product column
df_spark.select('Product').show()

In [ ]:
# Select multiple columns
df_spark.select(['Product', 'Country', 'Date']).show()

In [ ]:
# Check the types of the columns
df_spark.dtypes

In [ ]:
# Check the summary of the dataset
df_spark.describe().show()

In [12]:
# Add a new column
df_spark = df_spark.withColumn('New Column', df_spark['Boxes Shipped'] * 2)

In [13]:
# Drop the columns
df_spark = df_spark.drop('New Column')

In [ ]:
# Rename the columns
df_spark.withColumnRenamed('Sales Person', 'Salesperson').show()

## 2. Operations

In [ ]:
# Drop the missing values

# how parameter
df_spark.na.drop(how = 'any') # drop column if any missing value - default
df_spark.na.drop(how = 'all') # drop column if all missing value

# threshold parameter
df_spark.na.drop(thresh = 2) # specify the minimum number of non-null values requires for a row or column to be retained

# subset parameter
df_spark.na.drop(subset = ['Product']) # drop rows or columns with missing values in the specified columns

In [ ]:
# Fill the missing values
df_spark.na.fill('Missing Value') # fill all missing values with the specified value
df_spark.na.fill('Missing Value', subset = ['Product']).show() # fill missing values in the specified columns

In [ ]:
# Fill the missing values with the mean value
from pyspark.ml.feature import Imputer
from pyspark.sql.functions import collect_list, avg

df_spark = df_spark.withColumn("Boxes Shipped", df_spark["Boxes Shipped"].cast("double"))

imputer = Imputer(
    inputCols = ['Boxes Shipped'],
    outputCols = ["Boxes Shipped_imputed"]
    ).setStrategy("mean")

imputer.fit(df_spark).transform(df_spark).show()


## 3. Filter

In [18]:
# Casting the data type
from pyspark.sql.functions import regexp_replace
df_spark = df_spark.withColumn("Amount", regexp_replace(df_spark["Amount"], "[$,]", "").cast("double"))

In [ ]:
# Filter the dataset
df_spark.filter("Amount > 15000")

# Filter the dataset selecting the columns
df_spark.filter("`Boxes Shipped` < 300").select(['Sales Person', 'Country'])

# Filter the dataset using multiple conditions
df_spark.filter((df_spark['Amount'] > 15000) & 
                (df_spark['Country'] == 'UK')).show()

## 4. GroupBy and Aggregate Functions

In [ ]:
## GroupBy operation
df_spark.groupBy('Sales Person').sum().select(['Sales Person', 'sum(Boxes Shipped)']).show()

df_spark.groupBy('Country').mean().select(['Country', 'avg(Amount)']).show()

df_spark.groupBy('Country').count().show()

In [ ]:
df_spark.agg({'Amount': 'sum'}).show()